In [1]:
from probemapper.registration import ANTSPointTransformation, ANTSTransformationStages
from probemapper.io import save_nifti
import os, subprocess
import numpy as np
import pandas as pd

In [2]:
rootdir = "/bucket/ReiterU/DBS/data/20220613_P200_laqueus_DiI_L-A1/"
resultdir = os.path.join(rootdir, "analysis")
if not os.path.exists(resultdir):
    subprocess.run(["ssh", "deigo", "mkdir", resultdir])

In [3]:
cwd = os.path.basename(os.getcwd())
tmpdir = os.path.join("/flash/ReiterU/tmp/", cwd, f"indexing")
if not os.path.exists(tmpdir):
    os.makedirs(tmpdir)

### Transform channel coordinates

In [4]:
T = ANTSPointTransformation(
    ants_path="/apps/unit/ReiterU/ANTs/2.3.5/bin/",
    num_threads=8
)
T.set_transformations([
    ANTSTransformationStages(transformation=os.path.join(rootdir, "analysis/ants/F2M_0GenericAffine.mat"), inverse=True),
    ANTSTransformationStages(transformation=os.path.join(rootdir, "analysis/ants/F2M_1InverseWarp.nii.gz"), inverse=False),
])
T.set_moving_points(os.path.join(rootdir, "analysis/skeleton/channel_skeleton.csv"))
T.set_outname(os.path.join(tmpdir, "channel_LUT.csv"))

In [5]:
T.run()

['/apps/unit/ReiterU/ANTs/2.3.5/bin/antsApplyTransformsToPoints', '-d', '3', '-i', '/bucket/ReiterU/DBS/data/20220613_P200_laqueus_DiI_L-A1/analysis/skeleton/channel_skeleton.csv', '-o', '/flash/ReiterU/tmp/220617_P200_skeletonization/indexing/channel_LUT.csv', '-t', '[/bucket/ReiterU/DBS/data/20220613_P200_laqueus_DiI_L-A1/analysis/ants/F2M_0GenericAffine.mat,1]', '-t', '/bucket/ReiterU/DBS/data/20220613_P200_laqueus_DiI_L-A1/analysis/ants/F2M_1InverseWarp.nii.gz']



### Query region ID

In [6]:
from probemapper import skeletonization

In [7]:
df = pd.read_csv(os.path.join(tmpdir, "channel_LUT.csv"))
coords = np.array([df["Z"], df["Y"], df["X"]])

In [8]:
regions = skeletonization.query_region_id2(coords,
                     "/bucket/ReiterU/DBS/atlas/O_Laqueues_v0.1/Slicer_3D/Segmentation.seg.nrrd",
                     100, coords.shape[1]-100, 10)

In [9]:
df["Region"] = regions

In [10]:
df.insert(0, "channel", df.pop("channel"))

In [11]:
display(df)

,channel,X,Y,Z,X_org,Y_org,Z_org,Region
0,-100,2565.50,2691.55,2239.96,2292.70,3269.430,2319.59,8
1,-99,2566.08,2685.89,2245.41,2292.79,3259.850,2321.36,8
2,-98,2566.58,2680.32,2251.10,2292.87,3250.260,2323.12,8
3,-97,2566.86,2674.86,2257.36,2292.95,3240.680,2324.89,8
4,-96,2566.96,2669.50,2263.77,2293.03,3231.090,2326.66,8
...,...,...,...,...,...,...,...,...
304,204,2663.77,1242.07,4133.07,2375.47,443.090,3214.89,3
305,205,2661.37,1238.45,4139.38,2375.66,433.502,3218.13,3
306,206,2659.10,1234.48,4146.05,2375.87,423.909,3221.35,3
307,207,2656.90,1230.21,4153.08,2376.10,414.310,3224.55,3


In [12]:
df.to_csv(os.path.join(tmpdir, "channel_LUT.csv"), index=False)

In [13]:
sk = skeletonization.generate_skeleton_image(coords[:, 100:],
                             "/bucket/ReiterU/DBS/atlas/O_Laqueues_v0.1/Slicer_3D/Segmentation.seg.nrrd",
                             10)

In [14]:
sk = sk.astype("uint8")
save_nifti(sk, os.path.join(tmpdir, "probe_image.nii.gz"), 1, 1, 1)

In [15]:
subprocess.run(["scp", "-r", tmpdir, f"deigo:{resultdir}"])

CompletedProcess(args=['scp', '-r', '/flash/ReiterU/tmp/220617_P200_skeletonization/indexing', 'deigo:/bucket/ReiterU/DBS/data/20220613_P200_laqueus_DiI_L-A1/analysis'], returncode=0)